In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from selenium import webdriver
import main
import seaborn as sns
import chromedriver_autoinstaller
from datetime import datetime

In [ ]:
chromedriver_autoinstaller.install()

## Pull Data

In [ ]:
if __name__ == "__main__":
    driver = webdriver.Chrome()
    
# whoscored match centre url of the required match (Example: Barcelona vs Sevilla)
url = "https://www.whoscored.com/Matches/1640888/Live/England-Premier-League-2022-2023-Liverpool-Chelsea"
match_data = main.getMatchData(driver, url, close_window=True)

# Match dataframe containing info about the match
matches_df = main.createMatchesDF(match_data)

# Events dataframe      
events_df = main.createEventsDF(match_data)

# match Id
matchId = match_data['matchId']

# Information about respective teams as dictionary
home_data = matches_df['home'][matchId]
away_data = matches_df['away'][matchId]

home_name = home_data['name']
away_name = away_data['name']
date = match_data['startDate'].split('T')[0]
date_ = datetime.strptime(date, '%Y-%m-%d').date()
date_str = date_.strftime('%b %d %Y')

In [ ]:
events_df["passRecipient"] = events_df['playerName'].shift(-1)
events_df['teamRecipient'] = events_df['teamId'].shift(-1)
column_to_move = events_df.pop("passRecipient")
events_df.insert(26, "passRecipient", column_to_move)
column_to_move = events_df.pop("teamRecipient")
events_df.insert(27, "teamRecipient", column_to_move)

In [ ]:
homeid = home_data['teamId']
awayid = away_data['teamId']

In [ ]:
df_team = events_df[events_df['teamId'] == awayid]

defensive_actions = df_team[(df_team['type'].isin(['Tackle', 'Interception', 'BlockedPass', 'Aerial'])) 
                            ].reset_index(drop=True)
defensive_actions

In [ ]:
tackles = defensive_actions[(defensive_actions['type'] == 'Tackle')].reset_index(drop=True)
tackles_list = tackles.groupby(['playerName'])[['type']].count().reset_index().sort_values(by='type', ascending=False)[:3].reset_index(drop=True)
tackles_list

interceptions = defensive_actions[(defensive_actions['type'] == 'Interception')].reset_index(drop=True)
interceptions_list = interceptions.groupby(['playerName'])[['type']].count().reset_index().sort_values(by='type', ascending=False)[:3].reset_index(drop=True)
interceptions_list

bp = defensive_actions[(defensive_actions['type'] == 'BlockedPass')].reset_index(drop=True)
bp_list = bp.groupby(['playerName'])[['type']].count().reset_index().sort_values(by='type', ascending=False)[:3].reset_index(drop=True)
bp_list

aerials = defensive_actions[(defensive_actions['type'] == 'Aerial')].reset_index(drop=True)
aerials_list = aerials.groupby(['playerName'])[['type']].count().reset_index().sort_values(by='type', ascending=False)[:3].reset_index(drop=True)
aerials_list


In [ ]:
avg_action = defensive_actions['x'].mean()
avg_action = float(avg_action)
pos = avg_action - 1
avg_action_str = str(round(avg_action, 2))
pos

In [ ]:
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch
from matplotlib.colors import to_rgba
import matplotlib as mpl
import matplotlib.patheffects as path_effects
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import gaussian_filter
import matplotlib.colors
mpl.rcParams['figure.dpi'] = 800


norm=plt.Normalize(-2,2)
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["#404040","darkcyan"])


pitch = VerticalPitch(pitch_type='opta', pitch_color='#2B2B2B', line_color='#c7d5cc',pad_top=5, pad_left = 40,
                      line_zorder=2)
fig, axs = pitch.grid(figheight=10, title_height=0.08, endnote_space=0, axis=False, 
                      title_space=0, grid_height=0.82, endnote_height=0.05)

fig.set_facecolor("#2B2B2B")

# rectangle1 = plt.Rectangle((0, 0),110,100, color='darkcyan')
# plt.gca().add_patch(rectangle1, ax=axs['left'])

axs['title'].text(0, 0.55, away_name.upper() + '  - Duel Attempt Heatzone'.upper(), color='White',
                  va='center', ha='left', fontsize=17, fontname='Sans Serif')
axs['title'].text(0, 0.25, 'Where '.upper() + away_name.upper() + ' engaged in defensive actions'.upper(), color='White',
                  va='center', ha='left', fontsize=8)
axs['title'].text(0, 0.00, 'vs ' + home_name.upper() + '  |  ' + date_str.upper() + '  |  '  + 'CREATED BY ...', color='White',
                  va='center', ha='left', fontsize=8)

bin_statistic = pitch.bin_statistic(defensive_actions.x, defensive_actions.y, statistic='count', bins=(6, 5), normalize=True)
pitch.heatmap(bin_statistic, ax=axs['pitch'], cmap=cmap, edgecolor='#c7d5cc')
# labels = pitch.label_heatmap(bin_statistic, color='#f4edf0', fontsize=18,
#                              ax=axs['pitch'], ha='center', va='center',
#                              str_format='{:.0%}', fontname = 'Sans Serif')

axs['pitch'].annotate(text='            Tackles           '.upper(), xytext=(120, 98), xy=(60, 40), ha='center', 
                     va='center', color='white',
                     bbox=dict(facecolor='darkcyan', edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=8)

y = 95
i = 0
for p1, cnt in zip(tackles_list['playerName'], tackles_list['type']):
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    axs['pitch'].annotate(text=p1.upper(), 
                          xytext=(134, y - 2*i), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 6)
    axs['pitch'].annotate(text=cnt, 
                          xytext=(106, y - 2*i), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 6)
    
    i += 1

axs['pitch'].annotate(text='      Interceptions      '.upper(), xytext=(120, 83), xy=(60, 40), ha='center', va='center', color='white',
                     bbox=dict(facecolor='darkcyan', edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=8)
y = 80
i = 0
for p1, cnt in zip(interceptions_list['playerName'], interceptions_list['type']):
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    axs['pitch'].annotate(text=p1.upper(), 
                          xytext=(134, y - 2*i), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 6)
    axs['pitch'].annotate(text=str(cnt), 
                          xytext=(106, y - 2*i), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 6)
    
    i += 1

axs['pitch'].annotate(text='     Passes Blocked    '.upper(), xytext=(120, 68), xy=(60, 40), ha='center', va='center', color='white',
                     bbox=dict(facecolor='darkcyan', edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=8)
y = 65
i = 0
for p1, cnt in zip(bp_list['playerName'], bp_list['type']):
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    axs['pitch'].annotate(text=p1.upper(), 
                          xytext=(134, y - 2*i), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 6)
    axs['pitch'].annotate(text=cnt, 
                          xytext=(106, y - 2*i), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 6)
    
    i += 1

axs['pitch'].annotate(text='            Aerials            '.upper(), xytext=(120, 53), xy=(60, 40), ha='center', va='center', color='white',
                     bbox=dict(facecolor='darkcyan', edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=8)
y = 50
i = 0
for p1, cnt in zip(aerials_list['playerName'], aerials_list['type']):
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    axs['pitch'].annotate(text=p1.upper(), 
                          xytext=(134, y - 2*i), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 6)
    axs['pitch'].annotate(text=cnt, 
                          xytext=(106, y - 2*i), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 6)
    
    i += 1


avg_action = pitch.lines(avg_action, 0,
                         avg_action, 100, lw=2, color='white',
                         zorder=1, ax=axs['pitch'], linestyle='--')

axs['pitch'].annotate(text=str(avg_action_str) + '% UP PITCH', 
                          xytext=(1, pos-2), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 7)

axs['endnote'].set_xlim(0, 1)
axs['endnote'].set_ylim(0, 1)

axs['endnote'].text(0.51, 0.77, '> Attempted defensive duels \n> Does not have to win the ball back \n> Includes Tackles, Interceptions & Blocked Passes', color='#c7d5cc',
                    va='center', ha='left', fontsize=7, fontname = 'Sans Serif')

fig.savefig('defensiveAction.png', dpi=300)